## 제주도 관광 이슈 크롤링
네이버 뉴스에서 해당하는 달의 관광 이슈 크롤링

In [1]:
from ssl import SSLError
from urllib import parse
from urllib.error import URLError
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import socket
import random
import argparse
import datetime
import pandas as pd

In [14]:
def crawl(query, save_as, begin, end, sort=0, field=1, delay=0.5, timeout=30, page_limit=50):
    '''
    :param query: 네이버 '뉴스'란에서 검색할 검색어
    :param save_as: 검색 결과 저장 경로
    :param begin: '기간' -> 검색 기간 시작
    :param end: '기간' -> 검색 기간 끝
    :param sort: '유형' -> 0(관련도순) 1(최신순) 2(오래된순)
    :param field: '영역' -> 0(전체) 1(제목)
    :param delay: (옵션) 검색 리퀘스트 간격 (초)
    :param timeout: (옵션) 타임아웃 시 기다릴 시간 (초)
    :param page_limit: (옵션) 검색 결과에서 몇 페이지까지 갈 것인지 결정
    :return:
    '''

    # prerequisite
    df = pd.DataFrame(columns=['link', 'title', 'date', 'article'])

    # index settings
    # a single pages includes 10 news, starting from page 1 (index 1~10)
    current_index = 1
    max_index = 2

    while (current_index <= max_index) and (1 + current_index // 10 <= page_limit):
        print('\n' + 'crawling... %s (current_page / max_page %i/%i)' % (query, 1 + current_index // 10, 1 + max_index // 10))
        url = make_url(query, sort, field, begin, end, current_index)
        print('making url', url)

        print('making beautifulsoup object from html')
        bsobj = make_bsobj(url, delay, timeout, trial=10)

        if bsobj is None:
            continue
        print('extracting naver news urls from bsobj')
        naver_news_urls = make_naver_news_urls(bsobj)
        print(naver_news_urls)

        for url in naver_news_urls:
            print('\topening:', url)
            news_bsobj = BeautifulSoup(url, 'html.parser')

            if news_bsobj is None:
                continue

            attributes = get_attributes(news_bsobj)
            if attributes is None:
                continue

            date, article, title, newspaper = attributes
            df = df.append({'link': url,
                            'title': title,
                            'newspaper': newspaper,
                            'date': date,
                            'article': article}, ignore_index=True)
            print('\t', title)

        print('saving updated df')
        df = df.sort_values(by=['date'])
        df.to_excel(save_as, engine='xlsxwriter')

        print('updating current_news_index info')
        current_index += 10
        max_index = get_max_index(bsobj)
        if max_index is None:
            break

In [15]:
def make_url(query, sort, field, begin, end, page):
    url = "https://search.naver.com/search.naver?&where=news&query=" + parse.quote(query)
    url += "&sort=%i" % sort
    url += "&field=%i" % field
    url += "&ds=" + begin + "&de=" + end
    url += "&nso=so:r,p:"
    url += "from" + begin.replace(".", "") + "to" + end.replace(".", "")
    url += "&start=" + str(page)
    url += "&refresh_start=0"
    return url

In [16]:
def make_bsobj(url, delay=0.5, timeout=30, trial=10):
    ua = UserAgent(verify_ssl=False)
    count = 0

    while count < trial:
        try:
            time.sleep(delay + random.random())
            html = urlopen(Request(url=url, headers={'User-Agent': ua.random}), timeout=timeout)
            bsobj = BeautifulSoup(html, 'html.parser')
            return bsobj
        except (URLError, SSLError, socket.timeout) as e:
            print('(Error)', e)
            print('reloading...')
            count += 1
            time.sleep(timeout)
    return None

In [17]:
def make_naver_news_urls(bsobj):
    return [link['href'] for link in bsobj.find_all('a', 'news_tit')]

In [18]:
def get_attributes(bsobj):
    def _get_title(bsobj):
        title = bsobj.select('a', 'news_tit')[0].text
        title = title.encode('utf-8', 'replace').decode()
        print(title)
        return title

    def _get_article(bsobj):
        article = bsobj.select('#articleBodyContents')[0].text
        article = article.encode('utf-8', 'replace').decode()
        return article

    def _get_date(bsobj):
        splits = bsobj.select('.t11')[0].text.split(' ')
        date = splits[0] + ' ' + splits[2]
        date = datetime.datetime.strptime(date, '%Y.%m.%d. %H:%M')
        date += datetime.timedelta(hours=12 * int(splits[1] == '오후'))
        print(date)
        return date

    def _get_newspaper(bsobj):
        newspaper = bsobj.find("div", class_="press_logo").find('img', alt=True).get('alt')
        return newspaper

    try:
        return _get_date(bsobj), _get_article(bsobj), _get_title(bsobj), _get_newspaper(bsobj)
    except IndexError:
        print('(Error) crawling failed (maybe url is redirected to somewhere else)')
        return None

In [19]:
def get_max_index(bsobj):
    paging = bsobj.find("div", {"class": "sc_page_inner"})
    if not paging:
        print('(WARNING!) no results found')
        return None

    atags = paging.find_all('a')
    if not atags:
        print('(WARNING!) there is only one page')
        return None

    return max([int(atag["href"].split('start=')[1]) for atag in atags])

In [20]:
def get_arguments():
    # Argument configuration
    parser = argparse.ArgumentParser()
    parser.add_argument('--query', type=str, required=True, help='query to search on NAVER')
    parser.add_argument('--begin', type=str, required=True, help='crawling begin point (%%Y.%%m.%%d format)')
    parser.add_argument('--end', type=str, required=True, help='crawling end point (%%Y.%%m.%%d format)')
    parser.add_argument('--save_as', type=str, default='test2.xlsx', help='excel save path')
    parser.add_argument('--sort', type=int, default=0, help='search result sorting: 0(relevant), 1(newest), 2(oldest)')
    parser.add_argument('--field', type=int, default=1, help='search field: 0(all), 1(title)')
    return parser.parse_args()

In [22]:
crawl('제주도관광', 'test.xlsx', '2015.01.01', '2015.12.31')


crawling... 제주도관광 (current_page / max_page 1/1)
making url https://search.naver.com/search.naver?&where=news&query=%EC%A0%9C%EC%A3%BC%EB%8F%84%EA%B4%80%EA%B4%91&sort=0&field=1&ds=2015.01.01&de=2015.12.31&nso=so:r,p:from20150101to20151231&start=1&refresh_start=0
making beautifulsoup object from html
extracting naver news urls from bsobj
['http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151229_0010503813&cID=10813&pID=10800', 'http://www.nocutnews.co.kr/news/4524790', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=261210', 'http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151228_0010501924&cID=10813&pID=10800', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=261324', 'http://news.jtbc.co.kr/html/088/NB11132088.html', 'http://www.ablenews.co.kr/News/NewsContent.aspx?CategoryCode=0039&NewsCode=003920151223090111629787', 'http://www.nocutnews.co.kr/news/4522373', 'http://news.donga.com/3/all/20151221/75487308/1', 'http://www.newsis.com/ar_detail/vi

KeyboardInterrupt: 